In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240617/sc-E2G/test/results/GM12878/GM12878.multiome_7features/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240617/sc-E2G/test/results/GM12878/GM12878.multiome_7features/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/GM12878/1.prepare_data/1.seurat_pipeline.240615/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/GM12878/2.Genome_wide_prediction/Signac/Signac.250210/"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [6]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [7]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, 

Prepare background data

In [8]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 155270 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100028533-100029101      *
       [2]     chr1 100029136-100029435      *
       [3]     chr1 100034578-100034758      *
       [4]     chr1 100037728-100039259      *
       [5]     chr1 100046135-100047262      *
       ...      ...                 ...    ...
  [155266]     chrX       997541-998270      *
  [155267]     chrX     9988858-9989079      *
  [155268]     chrX       999293-999875      *
  [155269]     chrX     9995654-9996249      *
  [155270]     chrX     9997430-9997980      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [9]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100028533-100029101,57,31,50,34,42,30,6,40,37,24,⋯,30,36,33,34,56,41.65202,569,chr1,51,0.41927610
chr1-100029136-100029435,25,23,17,22,26,18,2,27,21,5,⋯,9,15,27,16,36,39.33333,300,chr1,16,0.02689509
chr1-100034578-100034758,13,5,14,14,9,2,0,11,12,5,⋯,16,12,10,19,18,41.43646,181,chr1,80,0.55015135
chr1-100037728-100039259,83,65,86,51,72,153,132,107,82,151,⋯,81,48,95,96,81,60.44386,1532,chr1,1457,0.96793972
chr1-100046135-100047262,53,31,67,81,55,48,11,91,52,46,⋯,86,71,81,106,197,39.09574,1128,chr1,417,0.86996844
chr1-100050188-100050426,20,12,17,14,12,4,3,19,17,10,⋯,16,15,12,22,26,41.84100,239,chr1,47,0.39281896
chr1-100064838-100065389,38,23,40,35,42,38,2,45,29,30,⋯,35,27,36,52,61,43.47826,552,chr1,26,0.16429445
chr1-100075645-100076196,33,21,34,34,37,30,4,51,29,26,⋯,39,23,45,56,59,44.56522,552,chr1,446,0.87673730
chr1-100076765-100077110,29,14,27,27,19,10,0,28,21,9,⋯,22,29,24,25,47,35.54913,346,chr1,238,0.79444194


Run Signac prediction

In [10]:
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 10129561 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100028533-100029101      * |         AGL
         [2]     chr1 100028533-100029101      * |       ALG14
         [3]     chr1 100028533-100029101      * |      CDC14A
         [4]     chr1 100028533-100029101      * |         DBT
         [5]     chr1 100028533-100029101      * |        DPH5
         ...      ...                 ...    ... .         ...
  [10129557]     chrX     9997430-9997980      * |      TCEANC
  [10129558]     chrX     9997430-9997980      * |        TLR7
  [10129559]     chrX     9997430-9997980      * |      TMSB4X
  [10129560]     chrX     9997430-9997980      * |     TRAPPC2
  [10129561]     chrX     9997430-9997980      * |        WWC3
                           PeakName               PairName    zscore    pvalue
                        <character>       

In [12]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna.rename,
                                 matrix.atac,
                                 meta.features,
                                 8)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 13.60808 hours

In [13]:
execution_time

Time difference of 13.60808 hours

Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 10129561 ranges and 5 metadata columns:
             seqnames              ranges strand |  TargetGene
                <Rle>           <IRanges>  <Rle> | <character>
         [1]     chr1 100028533-100029101      * |         AGL
         [2]     chr1 100029136-100029435      * |         AGL
         [3]     chr1 100034578-100034758      * |         AGL
         [4]     chr1 100037728-100039259      * |         AGL
         [5]     chr1 100046135-100047262      * |         AGL
         ...      ...                 ...    ... .         ...
  [10129557]     chrX   87338633-87338805      * |         CHM
  [10129558]     chrX   87664508-87664660      * |         CHM
  [10129559]     chrX   87896187-87896367      * |         CHM
  [10129560]     chrX   87900177-87900916      * |         CHM
  [10129561]     chrX   89871723-89871944      * |         CHM
                           PeakName               PairName    zscore    pvalue
                        <character>       

In [15]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100028533,100029101,569,*,AGL,chr1-100028533-100029101,chr1-100028533-100029101_AGL,-0.43720897,0.66902010,K562
chr1,100029136,100029435,300,*,AGL,chr1-100029136-100029435,chr1-100029136-100029435_AGL,-0.22335599,0.58837077,K562
chr1,100034578,100034758,181,*,AGL,chr1-100034578-100034758,chr1-100034578-100034758_AGL,-1.47839694,0.93034922,K562
chr1,100037728,100039259,1532,*,AGL,chr1-100037728-100039259,chr1-100037728-100039259_AGL,-1.42594077,0.92305728,K562
chr1,100046135,100047262,1128,*,AGL,chr1-100046135-100047262,chr1-100046135-100047262_AGL,-0.48186894,0.68505048,K562
chr1,100050188,100050426,239,*,AGL,chr1-100050188-100050426,chr1-100050188-100050426_AGL,0.16906167,0.43287407,K562
chr1,100064838,100065389,552,*,AGL,chr1-100064838-100065389,chr1-100064838-100065389_AGL,1.14109804,0.12691456,K562
chr1,100075645,100076196,552,*,AGL,chr1-100075645-100076196,chr1-100075645-100076196_AGL,0.90876305,0.18173761,K562
chr1,100076765,100077110,346,*,AGL,chr1-100076765-100077110,chr1-100076765-100077110_AGL,1.06632391,0.14313862,K562


In [16]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2                 dplyr_1.1.4                      
 [3] foreach_1.5.2                     Matrix_1.6